In [2]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import beta
import networkx as nx
%matplotlib inline

# Computing objectives

In [ ]:
# centers the opinion vector around 0
def mean_center(op, n):
    ones = np.ones((n, 1))
    x = op - (np.dot(np.transpose(op),ones)/n) * ones
    return x

# compute number of edges, m
def num_edges(L, n):
    m = 0
    for i in range(n):
        for j in range(n):
            if i > j and L[i,j] < 0:
                m += 1            
    return m

# maximizing (n/m)disagreement + polarization
def obj_sum(A, L, op, n, m):
    return obj_polarization(A, L, op, n, m) + (n / m)*obj_disagreement(A, L, op, n, m)
 
# maximizing polarization only: \bar{z}^T \bar{z}   
def obj_polarization(A, L, op, n, m):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean) 
    return np.dot(np.transpose(z_mean), z_mean)[0,0]          
    
# maximizing disagreement only: \bar{z}^T L \bar{z}
def obj_disagreement(A, L, op, n, m):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean)
    ans = np.dot(np.dot(np.transpose(z_mean), L), z_mean)[0,0]  
    return ans

In [ ]:
obj_functions = [obj_disagreement, obj_polarization, obj_sum]

# Generating adjacency matrices and opinions

In [ ]:
# constructs adjacency matrix for Erdos-Renyi model G(n, p)
def make_adj_matrix(l, p, weighted = True):
    # randomize upper triangle of the matrix 
    rand_count = int(0.5*(l**2 - l))
    weights = scipy.sparse.random(1, rand_count, density = p).A[0]    
    G = np.zeros((l, l))
    idx = 0
    for i in range(l):
        for j in range(l):
            if i == j:
                # diagonal is 0s
                continue
            elif i < j:
                if weighted:
                    G[i][j] = weights[idx]
                else:
                    if weights[idx] > 0:
                        G[i][j] = 1
                idx += 1
            else:
                # adjacency matrix is symmetric 
                G[i][j] = G[j][i]
                
    return G

In [ ]:
# create graph using stochastic block model
# l: length of side of matrix, number of vertices
# p1: density within the communities 
# p2: density of edges between the communities
# return arrays of vertices describing the partition into two communities, and adjacency matrix G
def make_block_matrix(l, p1, p2):
    # create two communities connected with density d1
    c1 = np.sort(np.random.choice(l, int(l/2), replace=False))
    l1 = len(c1)
    
    c2 = np.sort(list(set(np.arange(l)) - set(c1)))
    l2 = len(c2)

    weights1 = scipy.sparse.random(1, int(0.5*l1*(l1 - 1)), density=p1).A[0]
    weights2 = scipy.sparse.random(1, int(0.5*l2*(l2 - 1)), density=p1).A[0] 
    
    G = np.zeros((l, l))
    idx = 0
    for i in c1:
        for j in c1:
            if i == j:
                continue
            elif i < j:
                G[i][j] = weights1[idx]
                idx += 1
            else:
                G[i][j] = G[j][i]
    
    idx = 0
    for i in c2:
        for j in c2:
            if i == j:
                continue
            elif i < j:
                G[i][j] = weights2[idx]
                idx += 1
            else:
                G[i][j] = G[j][i]

      
    # weights for connections in between are of density d2
    idx = 0
    weights_between = scipy.sparse.random(1, l1*l2, density=p2).A[0]    
    for i in c1:
        for j in c2:
            G[i][j] = weights_between[idx]
            idx += 1        
    
    for i in c2:
        for j in c1:
            G[i][j] = G[j][i]
                
    return (c1, c2, G)

In [ ]:
# create innate opinion vector with community 1 ~ beta(a, b), community 2 ~ beta(b, a)
# a and b parametrize the beta distribution
# c1 and c2 describe the partition of vertices into communities using SBM
def make_beta_opinions(a, b, n, c1, c2):
    s1 = beta.rvs(a, b, size=int(n/2))
    s2 = beta.rvs(b, a, size=int(n/2))

    s = np.zeros((n, 1))
    idx1 = 0
    idx2 = 0
    for i in range(len(s)):
        if i in c1:
            s[i] = s1[idx1]
            idx1 += 1
        else:
            s[i] = s2[idx2]
            idx2 += 1
            
    return s

In [ ]:
# create graph using preferential attachment model
# n: number of vertices the final graph has
# n0: starting number of vertices
# init_G: the adjacency matrix of the graph to build on
def make_pref_attach(n, n0, m, init_G):
    # create array containing each vertex's (weighted) degree
    links = np.zeros(n)
    for i in range(n0):
        links[i] = np.sum(init_G[i, :])
        
    # create n x n adjacency matrix with existing init_G
    G = np.zeros((n, n))
    G[:n0, :n0] = init_G
        
    graph_size = n0
    for i in range(n0, n):        
        # choose m nodes to connect new node to
        vs = np.random.choice(graph_size, m, p = links[:graph_size]/sum(links[:graph_size]))
        
        
        # update adjacency matrix and links
        for v in vs:
            w = np.random.rand()
            G[i, v] = w
            G[v, i] = w
            
            links[i] += 1
            links[v] += 1
        
        graph_size += 1
        
    return G

# Visualizing networks

In [ ]:
# Visualizes the network of opinions and their edge weights, also coloring in which opinions should be changed.
def plot_network(G, op, combo, changed_opinions, initial):
    nxG = nx.from_numpy_matrix(G)
    
    pos = nx.drawing.layout.circular_layout(nxG)

    labels= {}
    colors= ["" for x in range(len(op))]
    
    for i in range(len(op)):
        labels[i] = str(round(op[i][0], 2))
        colors[i] = 'c'
      
    for j in range(len(combo)):
        colors[combo[j]] = 'm'
        if initial == True:
            x,y = pos[combo[j]]
            plt.text(x,y+0.15,s="s{}' = {}".format(combo[j] + 1, changed_opinions[j]),horizontalalignment='center')

    
    # pos=nx.spring_layout(nxG)
    nx.draw_networkx(nxG, pos, with_labels=True, labels=labels, node_color=colors, label="poop")
    # specify edge labels explicitly
    edge_labels=dict([((u,v,),round(d['weight'],3))
             for u,v,d in nxG.edges(data=True)])
    nx.draw_networkx_edge_labels(nxG, pos,edge_labels=edge_labels)

    plot = plt.axis('equal')

In [ ]:
# plot initial opinion graph and equilibrium graph side by side
def initial_vs_eq_networks(G, n, init_ops, idx, max_combos, changed_opinions):

    plt.figure(0)
    plot_network(G, init_ops, max_combos[idx], changed_opinions[idx], True)
    L = scipy.sparse.csgraph.laplacian(G, normed=False)

    s_copy = np.copy(init_ops)
    for i in range(len(max_combos[idx])):
        s_copy[max_combos[idx][i]] = changed_opinions[idx][i] 
    z= np.dot(np.linalg.inv((np.eye(n) + L)), s_copy)

    plt.figure(1)
    plot_network(G, z, max_combos[idx], changed_opinions[idx], False)

In [ ]:
# plot disagreement graphs
# The first network depicts the innate opinions. The blue nodes are those that should be adjusted 
# to be 0 or 1 as indicated. 
# The second network depicts the equilibrium state after the blue nodes have been altered.
def plot_disagreement(G, n, d, s, max_combos, changed_opinions, seed = None):
    print(obj_functions[0].__name__)
    if seed is None:
        initial_vs_eq_networks(G, n, s, 1, max_combos, changed_opinions)
    else:
        np.random.seed(seed)
        G = make_adj_matrix(n, d)
        s= np.random.rand(n, 1)
        initial_vs_eq_networks(G, n, s, 1, max_combos, changed_opinions)



In [ ]:
# plot polarization graphs
# The first network depicts the innate opinions. The blue nodes are those that should be adjusted 
# to be 0 or 1 as indicated. 
# The second network depicts the equilibrium state after the blue nodes have been altered.
def plot_polarization(G, n, d, s, max_combos, changed_opinions, seed = None):
    print(obj_functions[1].__name__)
    if seed is None:
        initial_vs_eq_networks(G, n, s, 0, max_combos, changed_opinions)
    else:
        np.random.seed(seed)
        G = make_adj_matrix(n, d)
        s= np.random.rand(n, 1)
        initial_vs_eq_networks(G, n, s, 0, max_combos, changed_opinions)


In [ ]:
# plot sum graphs
# The first network depicts the innate opinions. The blue nodes are those that should be adjusted 
# to be 0 or 1 as indicated. 
# The second network depicts the equilibrium state after the blue nodes have been altered.
def plot_sum(G, n, s, max_combos, changed_opinions):
    print(obj_functions[2].__name__)
    initial_vs_eq_networks(G, n, s, 2, max_combos, changed_opinions)
